# Descarga de datos geográficos de GeoElectoral
El Órgano Electoral mantiene datos geográficos públicos en una [instancia arcgis](https://geoelectoral.oep.org.bo/oep/rest/services/). A inicios de agosto de 2025, es posible acceder a coordenadas de asientos, recintos y circunscripciones electorales para las elecciones generales de 2020 y 2025, entre otra información. En este notebook descargamos los campos más esenciales (información geográfica + identificadores) de estas capas a documentos GeoPackage.

In [2]:
import requests
import geopandas as gpd
from os import makedirs
from tqdm import tqdm

Especificamos los datasets que queremos:

In [3]:
GEOELECTORAL = "https://geoelectoral.oep.org.bo/oep/rest/services"
DATA_PATHS = [
    {
        "year": 2020,  # el año, que utilizaremos como el nombre del directorio donde guardamos los datos
        "name": "asientos",  # el nombre, que utilizaremos como el filename del geopackage
        "item": "EleccionesGenerales2020/EleccionesNacionales2020",  # el item dentro del servidor arcgis
        "mapserver": 3,  # el mapserver dentro del item donde están los datos
        "fields": {
            "IdLoc": "asiento"
        },  # un diccionario que mapea los campos que queremos, aparte de la información geográfica, y los nombres con los que queremos almacenar esos campos
    },
    {
        "year": 2020,
        "name": "recintos",
        "item": "EleccionesGenerales2020/EleccionesNacionales2020",
        "mapserver": 2,
        "fields": {"IdLoc": "asiento", "Reci": "recinto"},
    },
    {
        "year": 2020,
        "name": "circunscripciones",
        "item": "Circunscripciones/CIRCUNSCRIPCIONES_BOLIVIA",
        "mapserver": 1,
        "fields": {"CU_2014": "circunscripcion"},
    },
    {
        "year": 2025,
        "name": "asientos",
        "item": "AsientosElectorales/AsientosRecintos_20_05_2025",
        "mapserver": 0,
        "fields": {"IdLoc": "asiento"},
    },
    {
        "year": 2025,
        "name": "recintos",
        "item": "AsientosElectorales/AsientosRecintos_20_05_2025",
        "mapserver": 1,
        "fields": {"IdLoc": "asiento", "Reci": "recinto"},
    },
    {
        "year": 2025,
        "name": "circunscripciones",
        "item": "EleccionesGenerales2020/Circunscripciones2025_4",
        "mapserver": 0,
        "fields": {"cu": "circunscripcion"},
    },
]

Y los descargamos.

In [1]:
def descargar_capa(datapath, output_crs=4326):
    print(f"Descargando {datapath['name']} para {datapath['year']}...")
    directory = datapath["year"]
    filename = f"{datapath['name']}.gpkg"
    makedirs(str(directory), exist_ok=True)

    features = []
    features_per_call = 100
    mapserver = f"{GEOELECTORAL}/{datapath['item']}/MapServer/{datapath['mapserver']}"
    crs = next(
        iter(
            requests.get(f"{mapserver}?f=json")
            .json()["sourceSpatialReference"]
            .values()
        )
    )
    fields = ",".join(datapath["fields"].keys())
    feature_ids = requests.get(
        f"{mapserver}/query?where=1=1&returnIdsOnly=true&f=json"
    ).json()["objectIds"]
    for i in tqdm(range(0, len(feature_ids), features_per_call)):
        features.extend(
            requests.get(
                f"{mapserver}/query",
                params={
                    "objectIds": ",".join(
                        map(str, feature_ids[i : i + features_per_call])
                    ),
                    "outFields": fields,
                    "returnGeometry": "true",
                    "f": "geojson",
                },
            ).json()["features"]
        )
    gdf = gpd.GeoDataFrame.from_features(features, crs=crs).rename(
        columns=datapath["fields"]
    )
    gdf.to_crs(output_crs).to_file(f"{directory}/{filename}", driver="GPKG")

In [4]:
for datapath in DATA_PATHS:
    descargar_capa(datapath)

Descargando asientos para 2020...


100%|██████████| 36/36 [00:19<00:00,  1.86it/s]


Descargando recintos para 2020...


100%|██████████| 54/54 [00:28<00:00,  1.87it/s]


Descargando circunscripciones para 2020...


...

100%|██████████| 38/38 [00:23<00:00,  1.64it/s]


Descargando recintos para 2025...


100%|██████████| 58/58 [00:34<00:00,  1.67it/s]


Descargando circunscripciones para 2025...


100%|██████████| 1/1 [00:17<00:00, 17.31s/it]
